In [82]:
import pandas as pd
import warnings
import firebase_admin
from firebase_admin import credentials, firestore
import numpy as np
from collections import defaultdict

import math

warnings.filterwarnings("ignore")

In [ ]:
cred = credentials.Certificate("slts-8e29d-firebase-adminsdk-i6d1u-d3ea7654ac.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [84]:
df = pd.read_csv("data_fire_11.csv")
df

,Timestamp,Which GROUP does the student belong to?,Student's Full Name,Student's Date of Birth (DOB),Student's Gender,District the student belongs to.,Name of the Samithi.,Student's Year of Joining Balvikas,Student's Date of Joining Balvikas (Optional),Has the student passed Group 2 exam?,...,Number of accompanying Mahilas(Women) who will need accommodation,Any food allergies for student? (Optional),Check In Date,Check In Time,Check out Date,Check out Time,Valid Group Selected based on DOB?,Valid Events Registered?,Overall Registration Status,Remarks
0,11/5/2024 13:45:27,Group 2,SAMYUTHA SRIRAM,"May 3, 2013",Female,Chennai South,Alwarpet,2022,"June 14, 2022",NaN,...,1.0,NaN,"December 25, 2024",6 AM,"December 25, 2024",830 PM,Accepted,Accepted,Accepted,
1,11/7/2024 22:24:42,Group 1,ANIRUDH SAI GUHAN .R,"March 19, 2016",Male,Coimbatore,Podanur,2020,"November 13, 2020",NaN,...,0.0,No,"December 25, 2024",6.30am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
2,11/8/2024 16:07:39,Group 1,G.LAKSHITH,"June 7, 2016",Male,Coimbatore,Saibaba colony,2022,"June 30, 2022",NaN,...,0.0,No,"December 25, 2024",6.30am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
3,11/8/2024 21:10:59,Group 1,SAI CHARANYA SRI P.S,"April 16, 2016",Female,Coimbatore,Podanur,2020,"June 5, 2020",NaN,...,0.0,NaN,"December 25, 2024",6.30 am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
4,11/10/2024 22:42:34,Group 2,S.SAI ESHWAR,"October 24, 2013",Male,Tiruvannamalai,Tiruvannamalai main samithi,2023,"June 18, 2023",NaN,...,1.0,No,"December 24, 2024",11 pm,"December 25, 2024",6:00pm,Accepted,Accepted,Accepted,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,12/4/2024 11:22:52,Group 2,KT. SRI SAI KANNAMMAI,"November 13, 2014",Female,Mayiladuthurai,MAYILADUTHURAI,2019,"May 5, 2019",NaN,...,1.0,0,"December 24, 2024",03.pm,"December 25, 2024",03.00pm,Accepted,Accepted,Accepted,
748,12/4/2024 11:47:06,Group 2,R.SIVADHARSHAN,"February 24, 2014",Female,Mayiladuthurai,SIRKALI,2021,"June 20, 2021",NaN,...,1.0,0,"December 24, 2024",08.00 pm,"December 25, 2024",03.00 pm,Accepted,Accepted,Accepted,
749,12/4/2024 12:00:32,Group 2,POOJAASRI A,"September 14, 2015",Female,Puducherry,KURUNJI NAGAR,2022,"June 5, 2022",NaN,...,15.0,NaN,"December 24, 2024",2.30 PM,"December 25, 2024",4.30 PM,Accepted,Accepted,Accepted,
750,12/6/2024 10:59:05,Group 3,DINESHKUMAR. S,"March 5, 2011",Male,Mayiladuthurai,SAIRKALI,2020,"June 20, 2020",Yes,...,1.0,0,"December 24, 2024",08.00 pm,"December 25, 2024",03.00 pm,Accepted,Accepted,Accepted,


In [85]:
# District

# df["District the student belongs to."].unique()

district = [
    "Chennai South",
    "Coimbatore",
    "Tiruvannamalai",
    "Tirunelveli",
    "Kanchipuram North",
    "Thanjavur",
    "Chennai South East",
    "Kanchipuram South",
    "Madurai",
    "Virudhunagar",
    "Trichy",
    "Dharmapuri / Krishnagiri",
    "Cuddalore",
    "Kanyakumari",
    "Chennai West",
    "Salem",
    "Nilgiris",
    "Tiruvallur East",
    "Tuticorin",
    "Tirupur",
    "Chennai North",
    "Sivaganga&Ramnad",
    "Erode",
    "Chennai North West",
    "Puducherry",
    "Vellore",
    "Dindigul",
    "Chennai East Coast",
    "Karur",
    "Theni",
    "Mayiladuthurai",
]

district_code = {}

for i in range(len(district)):
    district_code[district[i]] = str(i + 1).zfill(2)

In [86]:
def processData(df, district):
    # district_df = df
    district_df = df[df["District the student belongs to."] == district]
    # district_df = district_df[district_df["Overall Registration Status"] == "Accepted"]
    district_df["Timestamp"] = pd.to_datetime(district_df["Timestamp"])
    district_df = district_df.sort_values(by="Timestamp")
    district_df = district_df.reset_index(drop=True)
    district_df = district_df.drop(
        [
            "Timestamp",
            "Valid Group Selected based on DOB?",
            "Valid Events Registered?",
            # "Remarks",
        ],
        axis=1,
    )
    column_mapping = {
        "Which GROUP does the student belong to?": "studentGroup",
        "Student's Full Name": "studentFullName",
        "Student's Date of Birth (DOB)": "dateOfBirth",
        "Student's Gender": "gender",
        "District the student belongs to.": "district",
        "Name of the Samithi.": "samithiName",
        "Student's Year of Joining Balvikas": "yearOfJoiningBalvikas",
        "Student's Date of Joining Balvikas (Optional)": "dateOfJoiningBalvikas",
        "Has the student passed Group 2 exam?": "hasPassedGroup2Exam",
        "GROUP 1 - Select 1st event to register for.": "group1Event1",
        "GROUP 1 - Select 2nd event to register for. (Optional)": "group1Event2",
        "GROUP 1 - Select the group event to register for. (Optional)": "group1GroupEvent",
        "Select 1st event to register for.": "event1",
        "Select 2nd event to register for. (Optional)": "event2",
        'Select "Yes" to register for Quiz.': "isRegisteredForQuiz",
        "Date of arrival": "arrivalDate",
        "Time of arrival": "arrivalTime",
        "Does the student need pickup facility.": "needsPickup",
        "Mode of Travel": "modeOfTravel",
        "Pick Up Point": "pickupPoint",
        "Date of departure": "departureDate",
        "Time of departure": "departureTime",
        "Student needs drop facility.": "needsDrop",
        "Mode of Travel.1": "modeOfTravelForDrop",
        "Drop Off Point": "dropOffPoint",
        "Are adults accompanying the student?": "hasAccompanyingAdults",
        "Number of Gents(Men) accompanying student": "numMaleAccompanying",
        "Number of Mahilas(Women) accompanying student": "numFemaleAccompanying",
        "Number of non-participating siblings accompanying the student": "numNonParticipatingSiblings",
        "Name of the accompanying person": "accompanyingPersonName",
        "Gender of the accompanying person": "accompanyingPersonGender",
        "Relation": "accompanyingPersonRelation",
        "Contact Number of accompanying person": "accompanyingPersonContact",
        "Age of accompanying person": "accompanyingPersonAge",
        "Does the student need accommodation?": "needsAccommodation",
        "Number of accompanying Gents(Male) who will need accommodation": "numMaleAccompanyingNeedAccommodation",
        "Number of accompanying Mahilas(Women) who will need accommodation": "numFemaleAccompanyingNeedAccommodation",
        "Any food allergies for student? (Optional)": "foodAllergies",
        "Check In Date": "checkInDate",
        "Check In Time": "checkInTime",
        "Check out Date": "checkOutDate",
        "Check out Time": "checkOutTime",
        "Overall Registration Status": "overallRegistrationStatus",
        "Remarks": "remarks",
    }
    district_df = district_df.rename(columns=column_mapping)
    # district_df['dateOfBirth'] = pd.to_datetime(district_df['dateOfBirth']).strftime('%Y-%m-%d')
    # district_df['arrivalDate'] = pd.to_datetime(district_df['arrivalDate']).strftime('%Y-%m-%d')
    # district_df['departureDate'] = pd.to_datetime(district_df['departureDate']).strftime('%Y-%m-%d')
    # district_df['checkInDate'] = pd.to_datetime(district_df['checkInDate']).strftime('%Y-%m-%d')
    # district_df['checkOutDate'] = pd.to_datetime(district_df['checkOutDate'])
    return district_df

group_code = {
    "Group 1": "G1",
    "Group 2": "G2",
    "Group 3": "G3",
    "General Category": "G4",
}

gender_code = {"Male": "M", "Female": "F"}


def prepareDataForFirebase(df, district):
    df = processData(df, district)
    data = df.to_dict(orient="records")

    # Convert nan fields to empty strings
    for i in range(len(data)):
        for key in data[i].keys():
            if (
                key
                in [
                    "group1Event1",
                    "group1Event2",
                    "event1",
                    "event2",
                    "group1GroupEvent",
                ]
                and type(data[i][key]) == float
                and math.isnan(data[i][key])
            ):
                data[i][key] = ""

    for i in range(len(data)):
        if data[i]["studentGroup"] == "Group 1":
            if len(data[i]["group1Event1"]) > 0:
                data[i]["group1Event1"] = f"G1-{data[i]['group1Event1']}"

            if len(data[i]["group1Event2"]) > 0:
                data[i]["group1Event2"] = f"G1-{data[i]['group1Event2']}"

            data[i]["registeredEvents"] = [
                data[i]["group1Event1"],
                data[i]["group1Event2"],
                data[i]["group1GroupEvent"],
            ]

            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["hasPassedGroup2Exam"]

        elif data[i]["studentGroup"] == "Group 2":
            if "GROUP" not in data[i]["event1"] and len(data[i]["event1"]) > 0:
                data[i]["event1"] = f"G2-{data[i]['event1']}"
            if "GROUP" not in data[i]["event2"] and len(data[i]["event2"]) > 0:
                data[i]["event2"] = f"G2-{data[i]['event2']}"

            data[i]["registeredEvents"] = [data[i]["event1"], data[i]["event2"]]
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["hasPassedGroup2Exam"]

        elif data[i]["studentGroup"] == "Group 3":
            if "GROUP" not in data[i]["event1"] and len(data[i]["event1"]) > 0:
                data[i]["event1"] = f"G3-{data[i]['event1']}"
            if "GROUP" not in data[i]["event2"] and len(data[i]["event2"]) > 0:
                data[i]["event2"] = f"G3-{data[i]['event2']}"

            data[i]["registeredEvents"] = [data[i]["event1"], data[i]["event2"]]
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]

        else:
            data[i]["registeredEvents"] = []
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["hasPassedGroup2Exam"]

        data[i]["registeredEvents"] = [
            x for x in data[i]["registeredEvents"] if str(x) != ""
        ]
        data[i][
            "studentId"
        ] = f"{group_code[data[i]['studentGroup']]}{gender_code[data[i]['gender']]}{district_code[district]}{str(len(data[i]['registeredEvents'])).zfill(2)}{str(i+1).zfill(3)}"
        # print(data[i]['studentId'])

    return data

In [87]:
def addDataToFirebase(df, district):
    data = prepareDataForFirebase(df, district)
    for i in range(len(data)):
        db.collection("registrationData").document(data[i]["studentId"]).set(data[i])
        # print(f"Done: {data[i]['studentId']}")


def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        # print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

In [88]:
# Delete all documents in the collection
delete_collection(db.collection("registrationData"), 10)

for i in range(len(district)):
    addDataToFirebase(df, district[i])
    print(f"Done: {district[i]}")

Done: Chennai South
Done: Coimbatore
Done: Tiruvannamalai
Done: Tirunelveli
Done: Kanchipuram North
Done: Thanjavur
Done: Chennai South East
Done: Kanchipuram South
Done: Madurai
Done: Virudhunagar
Done: Trichy
Done: Dharmapuri / Krishnagiri
Done: Cuddalore
Done: Kanyakumari
Done: Chennai West
Done: Salem
Done: Nilgiris
Done: Tiruvallur East
Done: Tuticorin
Done: Tirupur
Done: Chennai North
Done: Sivaganga&Ramnad
Done: Erode
Done: Chennai North West
Done: Puducherry
Done: Vellore
Done: Dindigul
Done: Chennai East Coast
Done: Karur
Done: Theni
Done: Mayiladuthurai
